# Aula 11 &mdash; Visualização interativa de dados com Bokeh (Parte 2)

Renato Vimieiro

rv2 {em} cin.ufpe.br

abril 2017

## Introdução

Na aula anterior vimos como criar gráficos com a interface básica de Bokeh. Como vimos, a interface básica valoriza facilidade de geração de visualizações sobre customização. Se quisermos criar visualizações adaptadas ao contexto da aplicação em que estamos trabalhando, então devemos usar a interface intermediária, ajustando detalhes com alguns itens da interface avançada. Nesta aula veremos como criar gráficos com a interface intermediária `bokeh.plotting`.

Começamos por carregar algumas bibliotecas.

In [9]:
import pandas as pd
import numpy as np

import bokeh.charts as charts
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, push_notebook
from bokeh.models import (Range1d, NumeralTickFormatter, 
                          FixedTicker, Legend, DatetimeTickFormatter,
                          CategoricalColorMapper, PreText, BoxAnnotation)
from bokeh.palettes import Set1_7
from bokeh.layouts import gridplot
from bokeh.core.properties import field

output_notebook()

Loading BokehJS ...

## Dados

Para ilustrar o funcionamento da biblioteca, vamos utilizar os dados do IPCA de 8 regiões metropolitanas do país. Os dados foram obtidos do site do IBGE (http://seriesestatisticas.ibge.gov.br/lista_tema.aspx?op=0&de=52&no=11). Foram obtidas as séries históricas do IPCA mensais desde janeiro de 1991 até julho de 2016.

As séries estão dividas em vários arquivos. Dessa forma, teremos de carregá-los um a um para depois concatená-los em um único data frame.

In [10]:
x = ! ls ../2017.1/data/ibge/ipca*
print(x)

['../2017.1/data/ibge/ipca-reg-metropolitana-ago91_jul99.csv', '../2017.1/data/ibge/ipca-reg-metropolitana-jan12_jul16.csv', '../2017.1/data/ibge/ipca-reg-metropolitana-jul06_dez11.csv', '../2017.1/data/ibge/ipca-reg-metropolitanas-ago99_jun06.csv']


In [11]:
ipca = pd.concat([pd.read_csv(a,index_col=[0,1],header=0,sep='\t',na_values=['nan','-'],
                              decimal=',', ) for a in x], axis=1)
ipca.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 109 entries, (Belem - PA, 1,Alimentacao e bebidas) to (Sao Paulo - SP, Indice geral)
Columns: 306 entries, jan/91 to jun/06
dtypes: float64(306)
memory usage: 261.1+ KB


Como veremos, é mais fácil trabalhar com os dados no formato longo. Portanto, vamos convertê-los para o formato usando a função `pandas.melt`.

In [12]:
ipcalong = pd.melt(ipca.reset_index(),id_vars=['Regiao','OPCAO'],var_name='mes',value_name='ipca')
ipcalong.head(10)

,Regiao,OPCAO,mes,ipca
0,Belem - PA,"1,Alimentacao e bebidas",jan/91,21.08
1,Belem - PA,"2,Habitacao",jan/91,25.65
2,Belem - PA,"3,Artigos de residencia",jan/91,14.49
3,Belem - PA,"4,Vestuario",jan/91,9.54
4,Belem - PA,"5,Transporte e comunicacao",jan/91,17.18
5,Belem - PA,"5,Transportes",jan/91,NaN
6,Belem - PA,"6,Saude e cuidados pessoais",jan/91,29.30
7,Belem - PA,"7,Despesas pessoais",jan/91,31.70
8,Belem - PA,"8,Educacao",jan/91,NaN
9,Belem - PA,"9,Comunicacao",jan/91,NaN


As datas referentes aos índices estão no formato de string. Assim, devemos convertê-las para `datetime` antes de seguirmos a análise. Como os meses foram abreviados em português, precisamos mudar o `locale` para que a string de formatação de datas consiga interpretá-los corretamente.

In [13]:
import locale
locale.setlocale(locale.LC_ALL, '')

'pt_BR.UTF-8'

In [14]:
ipcalong.mes = pd.to_datetime(ipcalong.mes.str.title(),format='%b/%y')
ipcalong.ipca = ipcalong.ipca.values/100
ipcalong.head()

,Regiao,OPCAO,mes,ipca
0,Belem - PA,"1,Alimentacao e bebidas",1991-01-01,0.2108
1,Belem - PA,"2,Habitacao",1991-01-01,0.2565
2,Belem - PA,"3,Artigos de residencia",1991-01-01,0.1449
3,Belem - PA,"4,Vestuario",1991-01-01,0.0954
4,Belem - PA,"5,Transporte e comunicacao",1991-01-01,0.1718


## Tipo (fonte) de dados para Bokeh

Bokeh define um tipo de dados específico para ser usado nas visualizações. Os dados devem ser definidos como tabelas, em que cada coluna possui apenas valores escalares. A classe que define esse tipo é a `ColumnDataSource`, a qual pode ser importada de `bokeh.models`. O construtor da classe funciona aceita um dicionário ou um data frame como parâmetro. Os dados são armazenados internamente como um dicionário (`ColumnDataSource.data`). Esse dicionário será enviado ao backend de Javascript de Bokeh para ser usado nas visualizações.

In [15]:
from bokeh.models import ColumnDataSource

Vamos criar um conjunto de dados somente com os dados do IPCA geral (ele é composto por vários itens/categorias que são avaliadas separadamente).

In [16]:
source = ColumnDataSource(ipcalong[ipcalong.OPCAO == 'Indice geral'])

## Gramática dos gráficos

Bokeh segue uma ideia que tem sido bastante recorrente entre as diversas bibliotecas de visualização: a ideia de se especificar uma gramática para construir um gráfico. Um gráfico é uma área de desenho (canvas). Sobre essa área, elementos são agregados. Em outras palavras, agregam-se os eixos, marcadores, linhas, etc. Cada um desses elementos visuais possui suas propriedades que podem ser alteradas conforme a necessidade. Dessa forma, um gráfico é a sobreposição desses elementos visuais sobre a área de desenho.

A área de desenho é definida em `bokeh.plotting` pela função `figure`. Ela instancia essa área e a retorna como resultado da chamada.

In [17]:
p = figure(plot_height=400,plot_width=800,toolbar_location='above',
           tools="pan,box_select,tap,box_zoom,lasso_select,reset")

Uma vez que o gráfico tenha sido populado, pode-se visualizá-lo com a função `show`, tal qual fazíamos com os gráficos simples. De fato, eles são figuras em que certas propriedades são pré-definidas.

In [18]:
show(p)

Como vemos, uma vez que não adicionamos elementos visuais à figura, a área de desenho permanece vazia, exceto pelos elementos pré-definidos (barra de ferramentas).

Os objetos criados possuem diversos métodos para se adicionar os elementos visuais ao gráfico. Esses elementos são chamados de `Glyph` em Bokeh. Existe uma lista bem ampla de símbolos/marcadores que podem ser usados em Bokeh. Essa lista encontra-se no manual da biblioteca.

Em nosso primeiro exemplo, vamos criar um gráfico de dispersão acrescentando círculos à área de desenho.

In [19]:
p.circle('mes','ipca',source=source)
show(p)

Definimos no método que a coluna mês correspondia ao eixo x, enquanto a coluna ipca correspondia ao eixo y do conjunto de dados especificado em `source`. 

A figura gerada é, no entanto, pouco informativa, pois a informação da região não está presente. Além disso, os eixos estão formatados de maneira estranha. O eixo x, por exemplo, deveria exibir as datas e exibe valores absurdamente grandes. Assim, também corrigiremos a formatação dos eixos.

In [20]:
p = figure(plot_height=400,plot_width=800,toolbar_location='above',
           tools="pan,box_select,tap,box_zoom,lasso_select,reset")

p.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p.yaxis.formatter = NumeralTickFormatter(format='0.00%')

color_mapper = CategoricalColorMapper(
    factors = list(ipcalong.Regiao.unique()),
    palette = ['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', 
'#80b1d3', '#fdb462', '#b3de69', '#e41a1c',
'#377eb8', '#4daf4a', '#984ea3']
)

p.circle('mes','ipca',source=source,
         color={'field':'Regiao', 'transform':color_mapper},
         fill_alpha=0.3,
         legend=field('Regiao'))

show(p)

Bokeh permite criar interatividade inclusive com a legenda, definindo o comportamento que deve ocorrer quando clicamos sobre ela. Vamos definir que os dados sejam ocultados quando clicarmos.

Outra possibilidade que temos agora com a interface intermediária é de mover a legenda para fora da área de plotagem. Podemos colocá-la, por exemplo, no lado de fora à direita.

In [21]:
p.legend.click_policy = 'hide'
p.legend.location = (10,120)
p.right.append(p.legend[0])

show(p)

Infelizmente, como todos os itens foram gerados com a mesma chamada a `circle`, só é possível exibir ou ocultar todos de uma única vez. A solução para esse problema é criar os elementos um por vez e gerar uma legenda para eles em seguida.

Para produzir a visualização mencionada, precisamos voltar os dados para o formato amplo, onde as colunas serão os IPCAs nas regiões.

In [22]:
df = pd.pivot_table(ipcalong,columns='Regiao',values='ipca',index=['mes','OPCAO']).reset_index()
source = ColumnDataSource(df[df.OPCAO=='Indice geral'].drop('OPCAO',axis=1))

Precisamos criar um mapa de cores, uma vez que não usaremos mais o color mapper de Bokeh.

In [23]:
corRegiao = dict(zip(ipcalong.Regiao.unique(),['#8dd3c7', '#ffffb3', '#bebada', '#fb8072', 
'#80b1d3', '#fdb462', '#b3de69', '#e41a1c',
'#377eb8', '#4daf4a', '#984ea3']))
corRegiao

{'Belem - PA': '#8dd3c7',
 'Belo Horizonte - MG': '#ffffb3',
 'Curitiba - PR': '#bebada',
 'Fortaleza - CE': '#fb8072',
 'Grande Vitoria - ES': '#80b1d3',
 'Porto Alegre - RS': '#fdb462',
 'Recife - PE': '#b3de69',
 'Rio de Janeiro - RJ': '#e41a1c',
 'Salvador - BA': '#377eb8',
 'Sao Paulo - SP': '#4daf4a'}

In [24]:
p = figure(plot_height=400,plot_width=800,toolbar_location='above',
           tools="pan,box_select,tap,box_zoom,lasso_select,reset")

p.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p.yaxis.formatter = NumeralTickFormatter(format='0.00%')

glyphs = {d : [p.circle('mes',d,source=source,
                        color=corRegiao[d], fill_alpha=0.3)] 
          for d in ipcalong.Regiao.unique()}

legend = Legend(items=list(glyphs.items()), location=(10, 120))
legend.click_policy='hide'
p.add_layout(legend,'right')

show(p)

Podemos ainda acrescentar uma linha de tendência dos dados. Para isso, obtemos os coeficientes da reta usando regressão linear disponível em `numpy` (`numpy.polyfit`) e criamos uma função a partir dos mesmos com o auxílio de `numpy.poly1d`.

In [25]:
d = ipcalong[ipcalong.OPCAO=='Indice geral'].dropna()
coef = np.polyfit((d['mes'] - pd.datetime(1970,1,1)).dt.total_seconds(), d.ipca,deg=1)
func_ipca = np.poly1d(coef)
print(coef)

[ -2.04202954e-10   2.65492788e-01]


In [26]:
p.line(x=source.data['mes'],
       y=func_ipca((source.data['mes'] - pd.datetime(1970,1,1)).dt.total_seconds()),
      color='black')
show(p)

Podemos, finalmente, destacar os mandatos de cada presidente nesse período usando `BoxAnnotation` de Bokeh. Para isso, montamos uma tabela com os dados dos presidentes obtidos da Wikipedia e carregamos para um data frame.

In [27]:
presidentes = pd.read_csv('../2017.1/data/presidentes.csv',sep=';')
presidentes

,nome,inicio,fim,partido
0,Fernando Collor,15 de março de 1990,29 de dezembro de 1992,PRN
1,Itamar Franco,29 de dezembro de 1992,1 de janeiro de 1995,PMDB
2,Fernando Henrique Cardoso,1 de janeiro de 1995,1 de janeiro de 2003,PSDB
3,Luiz Inácio Lula da Silva,1 de janeiro de 2003,1 de janeiro de 2011,PT
4,Dilma Rousseff,1 de janeiro de 2011,31 de agosto de 2016,PT
5,Michel Temer,31 de agosto de 2016,NaN,PMDB


Ajustamos as datas para `datetime`.

In [28]:
presidentes.inicio = pd.to_datetime(presidentes.inicio,format='%d de %B de %Y')
presidentes.fim = pd.to_datetime(presidentes.fim,format='%d de %B de %Y')
presidentes.fim.fillna(ipcalong.mes.max(), inplace=True)
presidentes

,nome,inicio,fim,partido
0,Fernando Collor,1990-03-15,1992-12-29,PRN
1,Itamar Franco,1992-12-29,1995-01-01,PMDB
2,Fernando Henrique Cardoso,1995-01-01,2003-01-01,PSDB
3,Luiz Inácio Lula da Silva,2003-01-01,2011-01-01,PT
4,Dilma Rousseff,2011-01-01,2016-08-31,PT
5,Michel Temer,2016-08-31,2016-07-01,PMDB


Mapeamos os partidos a cores.

In [29]:
corPartido = dict(zip(presidentes.partido.unique(),['teal','orange','blue','red']))
corPartido

{'PMDB': 'orange', 'PRN': 'teal', 'PSDB': 'blue', 'PT': 'red'}

Criamos uma anotação para cada presidente.

In [30]:
boxes = presidentes.apply(lambda x: BoxAnnotation(left=x['inicio'].timestamp()*1000,\
                                                       right=x['fim'].timestamp()*1000,\
                                                       fill_color=corPartido[x['partido']],\
                                                       fill_alpha=0.1), \
                 axis=1).values


E adicionamos ao gráfico.

In [31]:
for b in boxes:
    p.add_layout(b)

show(p)    

Podemos ainda acrescentar a sigla do partido em cada região.

In [32]:
f = lambda x: (x['inicio'].timestamp()+(x['fim'].timestamp()+x['inicio'].timestamp())/2)*1000
from bokeh.models import Label
labels = presidentes.apply(lambda x: Label(text=x['partido'],x=f(x),y=0.7,text_font_size='10pt'),
                 axis=1).values

In [33]:
p.y_range = Range1d(0,0.7)

for l in labels:
    p.add_layout(l)

In [34]:
from bokeh.models import LabelSet
labelSource = ColumnDataSource(dict(x=presidentes['inicio'].map(lambda x: x.timestamp()*1000),
                                    y=np.repeat(0.55,presidentes.shape[0]),
                                   text=presidentes['partido']))
labels = LabelSet(x='x', y='y', text='text',
         x_offset=5, y_offset=5, source=labelSource, text_font_size='8pt')
p.add_layout(labels)

In [35]:
show(p)

Ainda que seja útil destacar os períodos de cada presidente, o usuário pode querer destacar um único período/presidente. Assim, seria interessante acrescentar alguns checkboxes à visualização que permitissem marcar ou desmarcar um período/presidente.

Essa funcionalidade requer o uso de um callback, que deve ser escrito em Javascipt, responsável por atualizar a visualização como resultado do evento de marcar ou desmarcar o checkbox. Veremos como criar tal interatividade.

In [36]:
from bokeh.models import CustomJS, CheckboxGroup
from bokeh.layouts import widgetbox

boxes = presidentes.apply(lambda x: BoxAnnotation(left=x['inicio'].timestamp()*1000,\
                                                       right=x['fim'].timestamp()*1000,\
                                                       fill_color=corPartido[x['partido']],\
                                                       fill_alpha=0.1), \
                 axis=1).values

code = '''\
for(i=0; i < ckbox.labels.length; i++){
    if(ckbox.active.indexOf(i)>-1){//i is active
       box = eval('box'+i)
       box.fill_alpha = 0.1
       box.line_alpha = 1    
    } else{
       box = eval('box'+i)
       box.fill_alpha = 0
       box.line_alpha = 0    
    }
}
'''
callback = CustomJS(code=code, args={})
checkbox_group = CheckboxGroup(
        labels=presidentes.nome.values.tolist(), active=list(range(len(boxes))), 
    callback=callback)

#toggle = Toggle(label="Red Box", button_type="success", callback=callback)
callback.args = {'ckbox': checkbox_group}
callback.args.update(dict(zip(['box{}'.format(a) for a in range(len(boxes))],boxes)))


In [37]:
p = figure(plot_height=400,plot_width=800,toolbar_location='below',
           tools="pan,box_select,tap,box_zoom,lasso_select,reset",
          toolbar_sticky=False)

p.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p.yaxis.formatter = NumeralTickFormatter(format='0.00%')

glyphs = {d : [p.circle('mes',d,source=source,
                        color=corRegiao[d], fill_alpha=0.3)] 
          for d in ipcalong.Regiao.unique()}
legend = Legend(items=list(glyphs.items()), location=(10, 50))
legend.click_policy='hide'
p.add_layout(legend,'right')

for b in boxes:
    p.add_layout(b)

show(gridplot([[p,checkbox_group]], 
              toolbar_location='below',
             sizing_mode="scale_width"))

Outro recurso interessante de Bokeh é a possibilidade de associar os eixos dos diversos gráficos da visualização. Podemos fazer com que a seleção de dados num gráfico seja refletida em outros gráficos da visualização compartilhando os eixos entre eles. No exemplo a seguir, plotaremos o índice geral do IPCA e os componentes de habitação, alimentação e educação em uma única visualização (quatro gráficos distintos). Vamos associar os eixos e dados desses gráficos de forma que a interação do usuário com qualquer deles seja refletida nos demais.

In [38]:
d2 = ipcalong[(ipcalong.Regiao=='Recife - PE')]
s = ColumnDataSource(pd.pivot_table(d2,columns='OPCAO',values='ipca',index='mes'))

In [39]:
p = figure(plot_width=400, plot_height=400, title='Geral',
           tools="pan,box_select,tap,box_zoom,lasso_select,reset")
p.line(x='mes', y='Indice geral',color="firebrick", source=s)
p.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p.yaxis.formatter = NumeralTickFormatter(format='0.00%')

p2 = figure(plot_width=400, plot_height=400, x_range = p.x_range, 
            y_range = p.y_range, title='Alimentação',
            tools="pan,box_select,tap,box_zoom,lasso_select,reset")
p2.line(x='mes', y='1,Alimentacao e bebidas',color="firebrick", source=s)
p2.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p2.yaxis.formatter = NumeralTickFormatter(format='0.00%')

p3 = figure(plot_width=400, plot_height=400, x_range = p.x_range, 
            y_range = p.y_range, 
            title='Habitação',
            tools="pan,box_select,tap,box_zoom,lasso_select,reset")
p3.line(x='mes', y='2,Habitacao',color="firebrick", source=s)
p3.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p3.yaxis.formatter = NumeralTickFormatter(format='0.00%')

p4 = figure(plot_width=400, plot_height=400, 
            x_range = p.x_range, y_range = p.y_range, 
            title='Educação',
            tools="pan,box_select,tap,box_zoom,lasso_select,reset")
p4.line(x='mes', y='8,Educacao',color="firebrick",source=s)
p4.xaxis.formatter = DatetimeTickFormatter(months='%b/%Y')
p4.yaxis.formatter = NumeralTickFormatter(format='0.00%')

show(gridplot([[p,p2],[p3,p4]]))